In [ ]:
from l2cs import Pipeline, render
import cv2
import os
import torch

In [ ]:
gaze_pipeline = Pipeline(
    weights='/mnt/c/Users/mjh/Downloads/L2CSNet_gaze360.pkl',
    arch='ResNet50',
    device=torch.device('cpu') # or 'gpu'
)

# vid_parent_dir = '/mnt/e/wsl_projects/LivePortrait/sync_output/2024-12-04-17-19_norm_no_vel_ep_60/cfg_0.65_mouth_0.25_subtract_False'
vid_parent_dir = '/mnt/e/data/diffposetalk_data/TFHP_raw/crop/TH_00192'
# vid_path = os.path.join(vid_parent_dir, 'audio_i8.wav_img_t4.jpg.mp4')
vid_path = os.path.join(vid_parent_dir, '000.mp4')
print(f"vid exists: {os.path.exists(vid_path)}")
cap = cv2.VideoCapture(vid_path)
_, frame = cap.read()

In [ ]:
frame.shape

In [ ]:
# Process frame and visualize
results = gaze_pipeline.step(frame)
frame = render(frame, results)

In [ ]:
# Import additional libraries for plotting
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Lists to store pitch and yaw values
pitch_values = []
yaw_values = []
frame_numbers = []

# Reset video capture to start
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Read all frames
for _ in tqdm(range(total_frames)):
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame
    results = gaze_pipeline.step(frame)

    # Store results
    pitch_values.append(results.pitch[0])
    yaw_values.append(results.yaw[0])
    frame_numbers.append(frame_count)
    frame_count += 1

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot pitch
ax1.plot(frame_numbers, pitch_values, 'b-')
ax1.set_title('Pitch over Time')
ax1.set_xlabel('Frame Number')
ax1.set_ylabel('Pitch (radians)')
ax1.grid(True)

# Plot yaw
ax2.plot(frame_numbers, yaw_values, 'r-')
ax2.set_title('Yaw over Time')
ax2.set_xlabel('Frame Number')
ax2.set_ylabel('Yaw (radians)')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Don't forget to release the video capture
cap.release()